## Exemplo de PINN com Equação de Poisson
#### Modelando o problema com redes neurais artificiais

### Iniciando o Ambiente

Será utilizado o PyTorch para a implementação da rede neural proposta como aprendizado básico.

In [ ]:
!pip install plotly --quiet

import torch
import torch.nn as nn
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1)

### Domínio e Condições de Contorno

O contorno será composto de 4 lados

In [ ]:
def gerar_pontos_contorno(pontos_no_contorno,comprimento_x, comprimento_y):
    pontos_por_lado = pontos_no_contorno//4

    # Lado 1 (x = 0, qualquer y)
    x_lado1 = 0 * np.ones((pontos_por_lado,1))
    y_lado1 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_y)

    u_lado1 = np.sin(2*np.pi*y_lado1/comprimento_x)

    # Lado 2 (x = comprimento_x, qualquer y)
    x_lado2 = comprimento_x * np.ones((pontos_por_lado,1))
    y_lado2 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_y)

    u_lado2 = np.sin(2*np.pi*y_lado2/comprimento_x)

    # Lado 3 (x = qualquer, y = 0)

    x_lado3 = np.random.uniform(size=(2*pontos_por_lado,1),low=0,high=comprimento_x)
    y_lado3 = 0 * np.ones((2*pontos_por_lado,1))

    u_lado3 = np.sin(2*np.pi*x_lado3/comprimento_x)

    # Lado 4 (x = qualquer, y = comprimento_y)
    x_lado4 = np.random.uniform(size=(pontos_por_lado, 1), low=0, high=comprimento_x)
    y_lado4 = comprimento_y * np.ones((pontos_por_lado, 1))

    u_lado4 = np.sin(2 * np.pi * x_lado4 / comprimento_x)

    # Juntar todos os lados
    x_todos = np.vstack((x_lado1, x_lado2, x_lado3, x_lado4))
    t_todos = np.vstack((y_lado1, y_lado2, y_lado3, y_lado4))
    u_todos = np.vstack((u_lado1, u_lado2, u_lado3, u_lado4))

    # Criar arrays X e Y
    X_contorno = np.hstack((x_todos, t_todos))
    Y_contorno = u_todos

    return X_contorno, Y_contorno

Agora, hora de gerar os pontos para a equação avaliar

In [ ]:
def gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y):
  x_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=comprimento_x)
  y_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=comprimento_y)

  X_equacao = np.hstack((x_dominio,y_dominio))

  return X_equacao

Vamos gerar os pontos para análise agora

In [ ]:
comprimento_x = 1
comprimento_y = 1

pontos_no_contorno = 600
pontos_no_dominio = 1000

X_contorno, Y_contorno = gerar_pontos_contorno(pontos_no_contorno,comprimento_x,comprimento_y)
X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y)


Vamos ver a vista superior do domínio

In [ ]:
# Vista superior
#fig = plt.figure()
#ax = fig.add_subplot()
scatter_contorno = px.scatter(x=X_contorno[:,0],y=X_contorno[:,1])
scatter_equacao = px.scatter(x=X_equacao[:,0],y=X_equacao[:,1], color_discrete_sequence=['red'])
fig = go.Figure(data=scatter_contorno.data+scatter_equacao.data)
fig.update_layout(xaxis_title='x',yaxis_title='t')
fig.show()

Agora, uma vista em perspectiva apenas dos pontos do contorno!

In [ ]:
# Vista em perspectiva
scatter_3d = px.scatter_3d(x=X_contorno[:,0].flatten(),y=X_contorno[:,1].flatten(),z=Y_contorno.flatten())
fig = go.Figure(scatter_3d)
fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))
fig.show()

### Definindo a Rede Neural

Como primeiro passo, vamos estruturá-la corretamente

In [ ]:
def criar_rede_neural(numero_de_neuronios):

  # Criar uma lista de todas as camadas
  camadas = []

  # Para cada camada, adicionar as conexões e a função de ativação
  for i in range(len(numero_de_neuronios)-1):
    camadas.append(nn.Linear(numero_de_neuronios[i],numero_de_neuronios[i+1]))
    camadas.append(nn.Tanh())

  # Remover a última camada, pois é a função de ativação
  camadas.pop()
  #camadas.pop()

  # Criar rede
  return nn.Sequential(*camadas)

A camada inicial deve ter dois perceptrons, que são nossas duas entradas. A última camada apenas um, que representa a nossa saída.

In [ ]:
numero_de_neuronios = [2, 20, 20, 20, 1]

rna = criar_rede_neural(numero_de_neuronios)

print(rna)

Sequential(
  (0): Linear(in_features=2, out_features=20, bias=True)
  (1): Tanh()
  (2): Linear(in_features=20, out_features=20, bias=True)
  (3): Tanh()
  (4): Linear(in_features=20, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)


### Escrevendo as funções de Perda

Primeiro, a perda referente às condições de contorno, a mais simples:

In [ ]:
def calc_perda_contorno(rna,X_contorno,Y_contorno):
  Y_predito = rna(X_contorno)
  return nn.functional.mse_loss(Y_predito, Y_contorno)

Agora, as perdas em relação à equação em si:

In [ ]:
def calc_residuo(rna, X_equacao, f):
    x = X_equacao[:, 0].reshape(-1, 1)
    y = X_equacao[:, 1].reshape(-1, 1)

    u = rna(torch.hstack((x, y)))

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

    # Uso da segunda derivada
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    laplacian_u = u_xx + u_yy

    residual = laplacian_u + f

    return residual


In [ ]:
def calc_perda_equacao(rna,X_equacao):

  residuo = calc_residuo(rna,X_equacao, torch.tensor(np.ones((pontos_no_dominio, 1)), dtype=torch.float32))

  return torch.mean(torch.square(residuo))

In [ ]:
def calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=0.2):

  perda_contorno = calc_perda_contorno(rna,X_contorno,Y_contorno)
  perda_equacao = calc_perda_equacao(rna,X_equacao)

  perda = (1-alpha)*perda_contorno + alpha*perda_equacao

  return perda, perda_contorno, perda_equacao

### Otimizador

Equilibrar as perdas em contorno e pela equação

In [ ]:
otimizador = torch.optim.Adam(rna.parameters(),lr=0.01)
agendador = torch.optim.lr_scheduler.StepLR(otimizador, step_size=1000, gamma=0.9)
alpha = 0.1

### Transferir para a GPU

Criar os tensores e passar para a GPU do Collab para rodar o problema

In [ ]:
X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float)
X_contorno = torch.tensor(X_contorno,dtype=torch.float)
Y_contorno = torch.tensor(Y_contorno,dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available () else 'cpu')
X_equacao = X_equacao.to(device)
X_contorno = X_contorno.to(device)
Y_contorno = Y_contorno.to(device)
rna = rna.to(device)

### Testes iniciais

Rodar algumas épocas para verificar a equivalência do sistema

In [ ]:
# Colocar rede em modo de treinamento
rna.train()
# FAZER ITERAÇÃO
for epoca in range(10):

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Mostrar resultados
  print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')

Epoca: 0, Perda: 0.5493004322052002 (Contorno: 0.49350446462631226, Equacao: 1.0514644384384155)
Epoca: 1, Perda: 0.5409016609191895 (Contorno: 0.4953951835632324, Equacao: 0.9504598379135132)
Epoca: 2, Perda: 0.5364179611206055 (Contorno: 0.48048368096351624, Equacao: 1.0398263931274414)
Epoca: 3, Perda: 0.5206477046012878 (Contorno: 0.47120168805122375, Equacao: 0.9656621217727661)
Epoca: 4, Perda: 0.5120205283164978 (Contorno: 0.4751240611076355, Equacao: 0.8440885543823242)
Epoca: 5, Perda: 0.49921536445617676 (Contorno: 0.4697917401790619, Equacao: 0.7640281915664673)
Epoca: 6, Perda: 0.4784083068370819 (Contorno: 0.45104122161865234, Equacao: 0.7247122526168823)
Epoca: 7, Perda: 0.46203795075416565 (Contorno: 0.43973198533058167, Equacao: 0.6627917289733887)
Epoca: 8, Perda: 0.43797603249549866 (Contorno: 0.42977195978164673, Equacao: 0.5118129849433899)
Epoca: 9, Perda: 0.4156794548034668 (Contorno: 0.42585429549217224, Equacao: 0.3241059482097626)


### Apresentar os resultados

Vamos plotar as soluções

In [ ]:
def calcular_grid(rna, comprimento_x, comprimento_y, nx=101, ny=101):
    # Define grid
    x = np.linspace(0., comprimento_x, nx)
    y = np.linspace(0., comprimento_y, ny)
    [y_grid, x_grid] = np.meshgrid(y, x)
    x = torch.tensor(x_grid.flatten()[:, None], requires_grad=True, dtype=torch.float).to(device)
    y = torch.tensor(y_grid.flatten()[:, None], requires_grad=True, dtype=torch.float).to(device)

    # Evaluate model
    rna.eval()
    Y_pred = rna(torch.hstack((x, y)))

    # Format results into an array
    u_pred = Y_pred.cpu().detach().numpy()[:, 0].reshape(x_grid.shape)

    return x_grid, y_grid, u_pred


In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, u_pred = calcular_grid(rna, comprimento_x, comprimento_y)

Agora, botando para rodar:

In [ ]:
numero_de_epocas = 20000
perda_historico = np.zeros(numero_de_epocas)
perda_contorno_historico = np.zeros(numero_de_epocas)
perda_equacao_historico = np.zeros(numero_de_epocas)
epocas = np.array(range(numero_de_epocas))

# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in epocas:

  # Resortear pontos
  #X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,tempo_final)
  #X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float).to(device)

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Guardar logs
  perda_historico[epoca] = perda.item()
  perda_contorno_historico[epoca] = perda_contorno.item()
  perda_equacao_historico[epoca] = perda_equacao.item()

  if epoca%500==0:
    print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')



Epoca: 0, Perda: 0.4014478921890259 (Contorno: 0.4202629029750824, Equacao: 0.2321130782365799)
Epoca: 500, Perda: 0.16207151114940643 (Contorno: 0.16478832066059113, Equacao: 0.1376202255487442)
Epoca: 1000, Perda: 0.08262886106967926 (Contorno: 0.07831459492444992, Equacao: 0.12145727127790451)
Epoca: 1500, Perda: 0.04419891908764839 (Contorno: 0.04153886064887047, Equacao: 0.0681394711136818)
Epoca: 2000, Perda: 0.03468646481633186 (Contorno: 0.029985252767801285, Equacao: 0.07699736207723618)
Epoca: 2500, Perda: 0.024606898427009583 (Contorno: 0.021111903712153435, Equacao: 0.05606186389923096)
Epoca: 3000, Perda: 0.01774260587990284 (Contorno: 0.014377694576978683, Equacao: 0.048026807606220245)
Epoca: 3500, Perda: 0.013404656201601028 (Contorno: 0.009922012686729431, Equacao: 0.044748447835445404)
Epoca: 4000, Perda: 0.01092556957155466 (Contorno: 0.006628307979553938, Equacao: 0.04960092529654503)
Epoca: 4500, Perda: 0.010077985003590584 (Contorno: 0.004841609392315149, Equacao:

Verificando os resultados

In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, z_pred = calcular_grid(rna, comprimento_x, comprimento_y)

# Plotar figura
fig = go.Figure(data=[go.Surface(x=x_grid, y=y_grid, z=z_pred)])

fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))